In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 15705, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 15705 (delta 10), reused 18 (delta 6), pack-reused 15672
Receiving objects: 100% (15705/15705), 14.44 MiB | 28.06 MiB/s, done.
Resolving deltas: 100% (10756/10756), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [5]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.0.1+cu118 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


In [6]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="zkyt15pyUkXSH9tlso0C")
project = rf.workspace("noor-djz41").project("multiuav")
dataset = project.version(4).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to MultiUav-4 in yolov5pytorch:: 100%|██████████| 19773/19773 [00:03<00:00, 5080.03it/s]


In [7]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

names:
- Fixed-wing
- Multi-rotor
- Single-rotor
nc: 3
roboflow:
  license: CC BY 4.0
  project: multiuav
  url: https://universe.roboflow.com/noor-djz41/multiuav/dataset/4
  version: 4
  workspace: noor-djz41
test: ../test/images
train: MultiUav-4/train/images
val: MultiUav-4/valid/images


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/hub/yolov5s-transformer.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/YOLOv5-CVIT-BiFPN.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer
# YOLOv5 backbone

  backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [512, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [512, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [512]],
   [-1, 1, Conv, [512, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [512]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [512, 3, 2]],  # 7-P5/32
   [-1, 3, C3TR, [512]],  # 9 <--- C3TR() Transformer module
   [-1, 1, SPPF, [512, 5]],  # 9
  ]

# YOLOv5 head
head:
   [[-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [256, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)
   

   [ -1, 1, Conv, [ 256, 3, 2 ] ],
   [ [ -1, 18, 4], 1, Concat, [ 1 ] ],  # cat head P3
   [-1, 1, SPP, [256, [5, 9, 13]]],
   [ -1, 3, C3, [256, False ] ],  # (P3/8-small)
   [-1, 1, CBAM, [256]],          # 28

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14, 6], 1, Concat, [1]],  # cat head P4
   [-1, 1, SPP, [256, [3, 7, 11]]],
   [-1, 3, C3, [256, False]],  #  (P4/16-medium) 
   [-1, 1, CBAM, [256]],       # 33

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 1, SPP, [256, [3, 5, 7]]],
   [-1, 3, C3TR, [256, False]],  #  (P5/32-large)
   [-1, 1, CBAM, [256]],        # 38

   [[28, 33, 38], 1, Detect, [nc,anchors]],  # Detect(P2, P3, P4, P5)
  ]

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch-size -1 --patience 50 --epochs 1000 --cos-lr --label-smoothing 0.01 --optimizer AdamW --hyp data/hyps/hyp.scratch-low.yaml --data {dataset.location}/data.yaml --cfg ./models/YOLOv5-CVIT-BiFPN.yaml --weights '' --project "/content/gdrive/MyDrive/Runs" --name YOLOv5-CVIT-BiFPN  --cache

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
!python train.py --img 416 --weights '' --project "/content/gdrive/MyDrive/Runs" --name YOLOv5-CVIT-BiFPN  --cache  --resume "/content/gdrive/MyDrive/Runs/YOLOv5-CVIT-BiFPN/weights/last.pt"

In [ ]:
#########Validate whole dataset
# Validate model and print evaluation metrics
!python val.py --weights runs/train/CVIT-BiFPN/weights/best.pt --data {dataset.location}/data.yaml --img 416 

In [ ]:
######## Test Valid image folder
%cd /content/yolov5/
!python detect.py --weights runs/train/CVIT-BiFPN/weights/best.pt --img 416 --conf 0.25 --source /content/yolov5/MultiUav-4/valid/images

In [ ]:
!python val.py --task study --weights runs/train/CVIT-BiFPN/weights/best.pt--data {dataset.location}/data.yaml --iou 0.5 

In [ ]:
!python val.py --task speed --weights runs/train/CVIT-BiFPN/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 

In [ ]:
##I have created a zip file:
!zip -r /content/CVIT-BiFPN.zip /content/yolov5/runs/
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/CVIT-BiFPN.zip")